In [ ]:
#!flask/bin/python
# -*- coding: utf-8 -*-

# for mysql & mongo
#import mysql.connector
#from mysql.connector import errorcode
from pymongo import MongoClient

# for api
from flask import Flask, jsonify, abort, make_response, request, render_template, Markup
from flask_cors import CORS
import json
import collections
import datetime
from dateutil.relativedelta import relativedelta
import urllib

# for country_info_templates
#import templates


app = Flask(__name__, template_folder='templates')

# for CORS
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/', methods=['GET'])
def index():
    return "Hello, world! This is for August POC"

@app.route('/api/test/', methods=['GET'])
def get_test():
	name = request.args.get('name', '', type=str); #type=str要去除掉，否則會有編碼問題
	print name;
	return name;

# potential projects
@app.route('/api/landing-map-markers/landing-map-markers/potential-projects/', methods=['GET'])
def get_potential_projects():
    # connect to mysql
    cnx = mysql.connector.connect(user='root', password='obor3',
                              host='obor-mysql',
                              database='tpdb',
                              charset='utf8')
    cursor = cnx.cursor()
    res = list()

    # get the parameter from args
    country_id = request.args.get('country_id', '', type=str).replace('"','').replace("'","")

    # sql script
    query_all = "SELECT tender_no, description, category, institute, location, country_id, longitude, latitude, advertised_date, closing_date, related_link FROM POTENTIALITEM;"
    #query2 = 'select tender_no, description, category, institute, location, country_id, longitude, latitude, advertised_date, closing_date, related_link FROM POTENTIALITEM where country_id =  %s;' % (country_id)
    query_country = "SELECT tender_no, description, category, institute, location, country_id, longitude, latitude, advertised_date, closing_date, related_link FROM POTENTIALITEM where country_id ='{0}';".format(country_id)
    
    # check the country_id value and then query the data
    if country_id:
        cursor.execute(query_country)
        alldata = cursor.fetchall()
        cursor.close()
        cnx.close()
        if alldata:
            return jsonify(alldata) 
        else:
            return jsonify({'error': 'County Not found'})
    else:
        cursor.execute(query_all)
        alldata = cursor.fetchall()
        cursor.close()
        cnx.close()
        return jsonify(alldata)

# potential projects
@app.route('/api/potential_item/', methods=['GET'])
def get_potential_items():
    # connection's parameter
    config = {
        'host': 'obor-mysql',
        'user': 'root', 
        'password': 'obor3',
        'charset': 'utf8'
    }

    # connect db
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as e:
        return jsonify({'error': 'Connect failed! -- ' + str(e)})
    else:
        pass

    # get the parameter from args
    country_id = request.args.get('country_id', '', type=str).replace('"','').replace("'","")

    # use request.args.get() got empty for chinese
    if 'industry_id' in request.args:
        industry_id = request.args['industry_id'].replace('"','').replace("'","").encode('utf8')

        # if industry_id == industry code, use mapping table
        industry_table = {
            "0": "制造业",
            "1": "交通运输",
            "10": "水利环境和公共",
            "11": "建筑业",
            "12": "批发和零售业",
            "13": "军事",
            "14": "卫生",
            "2": "金融",
            "3": "房地产",
            "4": "餐饮和住宿",
            "5": "居民服务",
            "6": "教育",
            "7": "文体",
            "8": "农林牧渔业",
            "9": "电力热力燃气",
            "999": "其他"
        }
        if industry_id.isdigit():
            if industry_id in industry_table:
                industry_id = industry_table[industry_id]
            else:
                return jsonify({"result": "industry code not founded"})

        # use two words for fuzzy query
        industry_id = unicode(industry_id.decode('utf-8'))[:2].encode('utf8') if len(unicode(industry_id.decode('utf-8'))) > 2 else industry_id
        
        # for 農林牧漁業
        if industry_id == '农林':
            industry_id = '农、'
    else:
        industry_id = ''

    # create query script
    query_filter = ''
    if country_id and industry_id:
        query_filter = "WHERE country.ID='{0}' AND pot.industry LIKE '%{1}%'".format(country_id, str(industry_id))
    else:
        if country_id:
            query_filter = "WHERE country.ID= '{0}'".format(country_id)
        if industry_id:
            query_filter = "WHERE pot.industry LIKE '%{0}%'".format(str(industry_id))
        
    query_script = """
        SELECT pot.item_name, pot.release_date, pot.item_type, pot.invest_type, pot.industry, pot.location, \
        pot.duration, pot.amount, pot.attract_investment_amount, pot.mark, pot.description, pot.person_name, \
        pot.organization, pot.position, pot.phone, pot.email, country.LATITUDE, country.LONGITUDE \
        FROM tpdb.POTENTIALITEMNEW AS pot \
        LEFT JOIN ciip.COUNTRY AS country \
        ON pot.location = country.NAME \
        %s \
        ORDER BY pot.release_date DESC \
    """ % (query_filter) 
            
    # return default change to dictionary
    cur = cnx.cursor(buffered=True, dictionary=True)

    # query from db
    cur.execute(query_script)
    res = cur.fetchall()
    cur.close()
    cnx.close()
    if res:
        return jsonify(res)
    else:
        return jsonify({"result": "NO DATA"})


# country info
@app.route('/api/country_info/<country>/<industry>')
def get_country_info(country, industry):
    # connect to mongodb and query data
    client = MongoClient('obor-mongo', 27017)
    data = client['obor']['obor_countryInfo'].find_one({"country_code": country})


    # check country exist
    if data is None:
        return jsonify({'error': 'County Not Found'})

    # check industry exist
    if industry not in data:
        return jsonify({'error': 'Industry Not Found'})

    # for disease
    def article_title_counts(keyword, location, limit_months=2):
        '''Return counts from articles which contain the keyword'''
        two_month_ago = (datetime.datetime.now()-relativedelta(months=limit_months)).strftime("%Y-%m-%d")
        count = client['obor']['obor_article'].count({
            "$and":[
                {
                    # filter title keywords
                    "_source.atitle": {
                        "$regex": ".*%s.*" % keyword, 
                        "$options": "-i"  # ignore case
                    }
                },
                
                {
                    # filter datetime
                    "_source.dateTime": {"$gt": "%s" % two_month_ago}
                },
                {
                    # filter country
                    "$or": [
                        {
                            "_source.location.country.wikiUri": {
                                "$regex": ".*%s.*" % location, 
                                "$options": "-i"  # ignore case
                            }
                        },
                        {
                            "_source.location.wikiUri": {
                                "$regex": ".*%s.*" % location, 
                                "$options": "-i"  # ignore case
                            }
                        },
                    ]
                }
            ]
        })
        return count
    # only society contain a dynamic infomation (disease)
    if  industry == "society":
        keywords = {
            "epidemic": u"疫情", 
            "diarrhea": u"腹泻", 
            "infectious": u"感染​​性疾病",
            "Hepatitis": u"肝炎",
            "Typhoid": u"伤寒", 
            "dengue": u"登革热",
            "malaria": u"疟疾", 
            "rabies": u"狂犬病",
            "poliomyelitis": u"脊髓灰质炎"
        }
        disease_res = list()

        # if counts > 0, we claim the disease existed
        for k, v in keywords.items():
            counts = article_title_counts(k, "pakistan")
            if counts:
                disease_res.append(v)
        data['society']['major_infectious_diseases_current'] = unicode(', '.join(disease_res))

    # map key with zh-simple
    with open("data/label_map.json", "r") as f:
        label_title = json.loads(f.read())

    # order the data for html layout
    ord_data = collections.OrderedDict(sorted(data[industry].items()))
    client.close()
    return render_template('countryInfo.html', data=ord_data, label = label_title)

# countryback info
@app.route('/api/countryback_info/<country>')
def get_countryback_info(country):
    # connection's parameter
    config = {
        'host': 'obor-mysql',
        'user': 'root', 
        'password': 'obor3',
        'database': 'tpdb',
        'charset': 'utf8'
    }

    # connect db
    try:
        cnx = mysql.connector.connect(**config)
    except mysql.connector.Error as e:
        return jsonify({'error': 'Connect failed! -- ' + str(e)})
    else:
        pass
    # return default change to dictionary
    cur = cnx.cursor(buffered=True, dictionary=True)
    # query from db
    cur.execute('SELECT * FROM COUNTRYBACK WHERE country_id = "%s";'% country)
    res = cur.fetchall()
    cur.close()
    cnx.close()

    # CHECK query data exist
    if res:
        return Markup(res[0]['background'])
    else:
        return jsonify({'error': 'Country Not Found'}) 


@app.errorhandler(404)
def not_found(error):
    '''Handle error code'''
    return make_response(jsonify({'error': 'Not Found'}), 404)



if __name__ == '__main__':
    # app.debug=True
    app.run(host="0.0.0.0")
